# Wegner Ising Gauge Theory (WIGT): Sampling, RG Learning, and String Tension

**Model.** Spins live on links: $\sigma_\ell \in \{\pm 1\}$. The Hamiltonian is the plaquette term
$$
H(\sigma) \;=\; -K \sum_{\square} \prod_{\ell \in \square} \sigma_\ell,
$$
with coupling $K$.

**Wilson loop.** For a closed contour $C$, define
$$
W(C)=\prod_{\ell\in C}\sigma_\ell.
$$
In a confining phase, $\langle W(C)\rangle \sim e^{-\sigma A(C)}$ (area law), where $\sigma$ is the **string tension**. In a deconfined phase, perimeter-law behavior dominates.

**Notebook goals.**
1. Implement sampling (Metropolis, Cluster) and basic diagnostics (histograms, ACF, ESS).
2. Learn $K$ from samples via RISE / pseudo-likelihood / ML approximations.
3. Perform multiple RG blocking schemes and estimate the flowed $K$.
4. Compute Wilson loops and extract the string tension $\sigma$ from area-law decay.

## 1. Lattice construction & utilities

We index **horizontal** (`hmap`) and **vertical** (`vmap`) links separately on an $n\times n$ periodic lattice.

In [ ]:
using Random
using Statistics
using StatsBase
using StatsPlots
using DataFrames
using Printf
using JuMP
using Ipopt
using MathOptInterface

# ---------- Lattice: link indices & plaquettes ----------

function link_indices(n::Int)
    hmap = Dict{Tuple{Int, Int}, Int}()
    vmap = Dict{Tuple{Int, Int}, Int}()
    idx = 1
    for i in 1:n, j in 1:n
        hmap[(i,j)] = idx; idx += 1
    end
    for i in 1:n, j in 1:n
        vmap[(i,j)] = idx; idx += 1
    end
    return (hmap, vmap)
end

function plaquettes(n::Int, hmap::Dict{Tuple{Int, Int}, Int}, vmap::Dict{Tuple{Int, Int}, Int})
    plaqs = Vector{NTuple{4, Int}}()
    for i in 1:n, j in 1:n
        top    = hmap[(i, j)]
        right  = vmap[(i, mod1(j+1, n))]
        bottom = hmap[(mod1(i+1, n), j)]
        left   = vmap[(i, j)]
        push!(plaqs, (top, right, bottom, left))
    end
    return plaqs
end

# ---------- Observables & diagnostics ----------

function average_plaquette_energy(samples::Matrix{Float64}, plaqlist::Vector{NTuple{4, Int}})
    total = 0.0
    num_conf, _ = size(samples)
    for p in plaqlist
        total += sum(samples[:, p[1]] .* samples[:, p[2]] .* samples[:, p[3]] .* samples[:, p[4]])
    end
    return total / (num_conf * length(plaqlist))
end

function compute_plaquette_energy_per_sample(samples::Matrix{Float64}, plaqlist::Vector{NTuple{4, Int}})
    num_conf, _ = size(samples)
    num_plaq = length(plaqlist)
    energies = zeros(Float64, num_conf)
    for p in plaqlist
        energies .+= samples[:, p[1]] .* samples[:, p[2]] .* samples[:, p[3]] .* samples[:, p[4]]
    end
    energies ./= num_plaq
    return energies
end

function compute_and_plot_acf(observable::Vector{Float64}; max_lag::Int=1000)
    acf_full = autocor(observable)
    acf_vals = acf_full[2:min(1+max_lag, length(acf_full))]
    plot(1:length(acf_vals), acf_vals, title="Autocorrelation Function", xlabel="Lag", ylabel="ACF", legend=false)
    return acf_vals
end

function estimate_autocorrelation_length(acf::Vector{Float64}; cutoff::Float64=0.05)
    valid_lags = findall(x -> abs(x) > cutoff, acf)
    if isempty(valid_lags); return 0.0; end
    last_valid = last(valid_lags)
    tau = 1.0 + 2.0 * sum(acf[1:last_valid])
    return tau
end

## 2. Sampling algorithms (**Metropolis** and **Cluster**)

We keep both for comparison. Other samplers (heat bath, PT, overrelaxation) can be added analogously.

In [ ]:
function metropolis_matrix(n::Int, plaqlist::Vector{NTuple{4, Int}},
                           K::Float64, nsteps::Int, nburn::Int, samples_matrix::Matrix{Float64};
                           step_interval::Int = n*n)
    num_samples = size(samples_matrix, 1)
    total_possible = floor(Int, (nsteps - nburn) / step_interval)
    if total_possible < num_samples
        @warn "Preallocated slots = $num_samples, collectable = $total_possible."
        num_samples = total_possible
    end

    nvars = 2n*n
    config = ones(Float64, nvars)

    link_plaquettes = [Int[] for _ in 1:nvars]
    for (pi, p) in enumerate(plaqlist)
        for l in p
            push!(link_plaquettes[l], pi)
        end
    end

    function energy_delta_for_flip(l)
        ΔEplaq = 0.0
        for pi in link_plaquettes[l]
            p = plaqlist[pi]
            prod_now = config[p[1]] * config[p[2]] * config[p[3]] * config[p[4]]
            ΔEplaq += (-prod_now) - prod_now
        end
        return -K * ΔEplaq
    end

    sample_idx = 1
    for sweep in 1:nsteps
        for l in randperm(nvars)
            dE = energy_delta_for_flip(l)
            if dE <= 0 || rand() < exp(-dE)
                config[l] *= -1
            end
        end
        if sweep > nburn && (sweep - nburn) % step_interval == 0
            if sample_idx <= size(samples_matrix,1)
                samples_matrix[sample_idx, :] = config
                sample_idx += 1
            else
                break
            end
        end
        if sweep % 50_000 == 0
            @printf("Metropolis: sweep %d / %d\n", sweep, nsteps)
        end
    end
    return samples_matrix
end

In [ ]:
function cluster_sampler(n::Int, plaqlist::Vector{NTuple{4, Int}},
                         K::Float64, nsteps::Int, nburn::Int,
                         samples_matrix::Matrix{Float64};
                         step_interval::Int=1)

    num_samples = size(samples_matrix, 1)
    nvars = 2n*n
    config = ones(Float64, nvars)

    # adjacency via shared plaquettes
    link_neighbors = [Set{Int}() for _ in 1:nvars]
    for p in plaqlist
        for i in 1:4, j in (i+1):4
            push!(link_neighbors[p[i]], p[j])
            push!(link_neighbors[p[j]], p[i])
        end
    end

    sample_idx = 1
    for sweep in 1:nsteps
        visited = falses(nvars)
        clusters = Vector{Vector{Int}}()
        for start in 1:nvars
            if visited[start]; continue; end
            cl = Int[]
            queue = [start]
            visited[start] = true
            while !isempty(queue)
                l = popfirst!(queue)
                push!(cl, l)
                for nb in link_neighbors[l]
                    if !visited[nb] && config[nb] == config[l]
                        p_bond = 1.0 - exp(-2K)
                        if rand() < p_bond
                            visited[nb] = true
                            push!(queue, nb)
                        end
                    end
                end
            end
            push!(clusters, cl)
        end
        for cl in clusters
            if rand() < 0.5
                for l in cl; config[l] *= -1; end
            end
        end

        if sweep > nburn && (sweep - nburn) % step_interval == 0
            if sample_idx <= num_samples
                samples_matrix[sample_idx, :] = config
                sample_idx += 1
            else
                break
            end
        end
        if sweep % 10_000 == 0
            @printf("Cluster: sweep %d / %d\n", sweep, nsteps)
        end
    end
    return samples_matrix
end

## 3. Learning the coupling \(K\)

We provide multiple estimators: **RISE**, normalized RISE (stabilized), **pseudo-likelihood**, and a simple **ML** surrogate.

In [ ]:
using Base.Threads
const MOI = MathOptInterface

function multiRISE_links_only_direct(samples::Matrix{Float64},
                                     plaqlist::Vector{NTuple{4, Int}};
                                     lambda::Float64=0.0)
    num_conf, _ = size(samples)
    s = zeros(Float64, num_conf)
    @inbounds for p in plaqlist
        s .+= samples[:, p[1]] .* samples[:, p[2]] .* samples[:, p[3]] .* samples[:, p[4]]
    end

    model = Model(Ipopt.Optimizer)
    set_optimizer_attribute(model, "print_level", 0)
    @variable(model, x >= 0.0)
    @NLobjective(model, Min, (1/num_conf)*sum(exp.(-x * s[k]) for k in 1:num_conf) + lambda * x)
    optimize!(model)
    return value(x)
end

function multiRISE_normalized(samples::Matrix{Float64},
                              plaqlist::Vector{NTuple{4,Int}}; λ::Float64=0.0)
    N, _ = size(samples)
    P = length(plaqlist)
    s = zeros(Float64, N)
    @inbounds for (t,r,b,l) in plaqlist
        s .+= samples[:,t] .* samples[:,r] .* samples[:,b] .* samples[:,l]
    end
    s ./= P
    model = Model(Ipopt.Optimizer)
    set_optimizer_attribute(model, "print_level", 0)
    @variable(model, x >= 0.0)
    @NLobjective(model, Min, (1/N)*sum(exp(-x * s[k]) for k in 1:N) + λ*x^2)
    optimize!(model)
    return value(x)
end

function PL_links(samples::Matrix{Float64}, plaqlist::Vector{NTuple{4, Int}}; lambda::Float64=0.0)
    num_conf, _ = size(samples)
    s = zeros(Float64, num_conf)
    @inbounds for p in plaqlist
        s .+= samples[:, p[1]] .* samples[:, p[2]] .* samples[:, p[3]] .* samples[:, p[4]]
    end
    model = Model(Ipopt.Optimizer)
    set_optimizer_attribute(model, "print_level", 0)
    @variable(model, x)
    @constraint(model, x >= 0)
    @NLobjective(model, Min, (1/num_conf)*sum(log(1 + exp.(-2 * x * s[k])) for k in 1:num_conf))
    optimize!(model)
    return value(x)
end

function robust_RISE_estimator(samples::Matrix{Float64}, plaqlist::Vector{NTuple{4, Int}}; verbose::Bool=true)
    num_conf, _ = size(samples)
    P = length(plaqlist)
    s = zeros(Float64, num_conf)
    @inbounds for (t,r,b,l) in plaqlist
        s .+= samples[:,t] .* samples[:,r] .* samples[:,b] .* samples[:,l]
    end
    s ./= P
    best_K, best_obj = 0.0, Inf
    for init_K in (0.1, 0.5, 1.0, 2.0, 3.0)
        model = Model(Ipopt.Optimizer)
        set_optimizer_attribute(model, "print_level", 0)
        set_optimizer_attribute(model, "max_iter", 1000)
        @variable(model, 0.001 <= x <= 10.0, start=init_K)
        @NLobjective(model, Min, (1/num_conf)*sum(exp(-x * s[k]) for k in 1:num_conf) + 1e-4*x^2)
        optimize!(model)
        if termination_status(model) in (MOI.OPTIMAL, MOI.LOCALLY_SOLVED)
            obj = objective_value(model)
            if obj < best_obj
                best_obj, best_K = obj, value(x)
            end
        end
    end
    verbose && println("Best RISE K = ", best_K)
    return best_K
end

function ML_estimator_plaquette(samples::Matrix{Float64}, plaqlist::Vector{NTuple{4, Int}}; verbose::Bool=true)
    avg_plaq = 0.0
    for p in plaqlist
        avg_plaq += mean(samples[:, p[1]] .* samples[:, p[2]] .* samples[:, p[3]] .* samples[:, p[4]])
    end
    avg_plaq /= length(plaqlist)
    model = Model(Ipopt.Optimizer)
    set_optimizer_attribute(model, "print_level", 0)
    @variable(model, 0.001 <= K_est <= 5.0, start=1.0)
    @NLobjective(model, Min, (avg_plaq - tanh(K_est))^2)
    optimize!(model)
    K_ml = value(K_est)
    if avg_plaq > 0.9
        K_ref = atanh(avg_plaq) * 1.1
        verbose && println("ML basic K = $K_ml, refined = $K_ref")
        return K_ref
    end
    return K_ml
end

## 4. RG coarse-graining schemes

We include multiple blocking maps (overlapping, majority, 4×4, decimation, checkerboards) and RG drivers that estimate flowed \(K\) via RISE.

In [ ]:
function coarse_plaquettes_4x4(samples_matrix::Matrix{Float64}, (h,v), n::Int)
    new_n = div(n,2)
    sizeN = size(samples_matrix,1)
    s_new = Matrix{Float64}(undef, sizeN, 2*new_n*new_n)
    (h_new,v_new) = link_indices(new_n)
    new_plaqs = plaquettes(new_n, h_new, v_new)
    for k in 0:2:new_n-1, l in 0:2:new_n-1
        s_new[:, h_new[mod1(k+1,new_n), mod1(l+1,new_n)]] =
            samples_matrix[:, h[mod1(4k+1,n), mod1(4l+1,n)]] .* samples_matrix[:, h[mod1(4k+1,n), mod1(4l+2,n)]] .* samples_matrix[:, h[mod1(4k+1,n), mod1(4l+3,n)]]
        s_new[:, h_new[mod1(k+2,new_n), mod1(l+1,new_n)]] =
            samples_matrix[:, h[mod1(4k+4,n), mod1(4l+1,n)]] .* samples_matrix[:, h[mod1(4k+4,n), mod1(4l+2,n)]] .* samples_matrix[:, h[mod1(4k+4,n), mod1(4l+3,n)]]
        s_new[:, v_new[mod1(k+1,new_n), mod1(l+1,new_n)]] =
            samples_matrix[:, v[mod1(4k+1,n), mod1(4l+1,n)]] .* samples_matrix[:, v[mod1(4k+2,n), mod1(4l+1,n)]] .* samples_matrix[:, v[mod1(4k+3,n), mod1(4l+1,n)]]
        s_new[:, v_new[mod1(k+1,new_n), mod1(l+2,new_n)]] =
            samples_matrix[:, v[mod1(4k+1,n), mod1(4l+4,n)]] .* samples_matrix[:, v[mod1(4k+2,n), mod1(4l+4,n)]] .* samples_matrix[:, v[mod1(4k+3,n), mod1(4l+4,n)]]
        s_new[:, h_new[mod1(k+1,new_n), mod1(l+2,new_n)]] =
            samples_matrix[:, h[mod1(4k+1,n), mod1(4l+4,n)]] .* samples_matrix[:, v[mod1(4k+2,n), mod1(4l+4,n)]]
        s_new[:, h_new[mod1(k+2,new_n), mod1(l+2,new_n)]] =
            samples_matrix[:, h[mod1(4k+3,n), mod1(4l+4,n)]] .* samples_matrix[:, h[mod1(4k+4,n), mod1(4l+4,n)]]
        s_new[:, v_new[mod1(k+2,new_n), mod1(l+1,new_n)]] =
            samples_matrix[:, v[mod1(4k+4,n), mod1(4l+1,n)]] .* samples_matrix[:, v[mod1(4k+4,n), mod1(4l+2,n)]]
        s_new[:, v_new[mod1(k+2,new_n), mod1(l+2,new_n)]] =
            samples_matrix[:, v[mod1(4k+4,n), mod1(4l+3,n)]] .* samples_matrix[:, v[mod1(4k+4,n), mod1(4l+4,n)]]
    end
    return s_new, new_plaqs, (h_new,v_new), new_n
end

function coarse_plaquettes_overlap(s::Matrix{Float64}, (h,v), n::Int)
    new_n = div(n,2)
    sizeN = size(s,1)
    s_new = Matrix{Float64}(undef, sizeN, 2*new_n*new_n)
    (h_new,v_new) = link_indices(new_n)
    new_plaqs = plaquettes(new_n, h_new, v_new)
    for k in 1:new_n, l in 1:new_n
        s_new[:, h_new[mod1(k,new_n), mod1(l,new_n)]] =
            (s[:, h[mod1(2k-1,n), mod1(2l-1,n)]] .* s[:, h[mod1(2k,n), mod1(2l-1,n)]])
        s_new[:, v_new[mod1(k,new_n), mod1(l,new_n)]] =
            (s[:, v[mod1(2k-1,n), mod1(2l-1,n)]] .* s[:, v[mod1(2k-1,n), mod1(2l,n)]])
    end
    return s_new, new_plaqs, (h_new,v_new), new_n
end

function coarse_plaquettes_sum(s::Matrix{Float64}, (h, v), n::Int)
    new_n = div(n, 2)
    sizeN = size(s,1)
    s_new = Matrix{Float64}(undef, sizeN, 2*new_n*new_n)
    (h_new, v_new) = link_indices(new_n)
    new_plaqs = plaquettes(new_n, h_new, v_new)
    for k in 1:new_n, l in 1:new_n
        hor_sum = s[:, h[mod1(2k-1, n), mod1(2l-1, n)]] .+ s[:, h[mod1(2k, n), mod1(2l-1, n)]]
        s_new[:, h_new[mod1(k,new_n), mod1(l,new_n)]] =
            map(x -> x > 0 ? 1.0 : x < 0 ? -1.0 : rand(Bool) ? 1.0 : -1.0, hor_sum)
        ver_sum = s[:, v[mod1(2k-1, n), mod1(2l-1, n)]] .+ s[:, v[mod1(2k-1, n), mod1(2l, n)]]
        s_new[:, v_new[mod1(k,new_n), mod1(l,new_n)]] =
            map(x -> x > 0 ? 1.0 : x < 0 ? -1.0 : rand(Bool) ? 1.0 : -1.0, ver_sum)
    end
    return s_new, new_plaqs, (h_new, v_new), new_n
end

# optional decimation-style
weight(u,K) = exp(K*u)

function coarse_grain_2d(s::Matrix{Float64}, (h, v), n::Int, K::Float64)
    new_n = n ÷ 2
    s_new = Matrix{Float64}(undef, size(s,1), 2*new_n^2)
    (h_new, v_new) = link_indices(new_n)
    for Kb in 1:new_n, L in 1:new_n
        sum_val = zeros(size(s,1))
        for u in (1.0, -1.0)
            s_left  = s[:, h[mod1(2*Kb-1, n), mod1(2*L-1, n)]]
            s_right = s[:, h[mod1(2*Kb-1, n), mod1(2*L,   n)]]
            sum_val .+= weight(u,K) * (s_left .* u .* s_right)/(2*cosh(K))
        end
        s_new[:, h_new[mod1(Kb,new_n), mod1(L,new_n)]] = sum_val
    end
    for Kb in 1:new_n, L in 1:new_n
        sum_val = zeros(size(s,1))
        for u in (1.0, -1.0)
            s_bottom = s[:, v[mod1(2*Kb-1, n), mod1(2*L-1, n)]]
            s_top    = s[:, v[mod1(2*Kb,   n), mod1(2*L-1, n)]]
            sum_val .+= weight(u,K) * (s_bottom .* u .* s_top)/(2*cosh(K))
        end
        s_new[:, v_new[mod1(Kb,new_n), mod1(L,new_n)]] = sum_val
    end
    new_plaqs = plaquettes(new_n, h_new, v_new)
    return s_new, new_plaqs, (h_new, v_new), new_n
end

# drivers
function RG_coarse_4x4(s::Matrix{Float64},(h,v),n::Int,n_steps::Int,K::Float64)
    xs = [K]
    for _ in 1:n_steps
        s, new_plaqs, (h,v), n = coarse_plaquettes_4x4(s,(h,v),n)
        push!(xs, multiRISE_links_only_direct(s, new_plaqs))
    end
    return xs
end

function RG_coarse_overlap(s::Matrix{Float64},(h,v),n::Int,n_steps::Int,K::Float64)
    xs = [K]
    for _ in 1:n_steps
        s, new_plaqs, (h,v), n = coarse_plaquettes_overlap(s,(h,v),n)
        push!(xs, multiRISE_links_only_direct(s, new_plaqs))
    end
    return xs
end

function RG_coarse_sum(s::Matrix{Float64},(h,v),n::Int,n_steps::Int,K::Float64)
    xs = [K]
    for _ in 1:n_steps
        s, new_plaqs, (h,v), n = coarse_plaquettes_sum(s,(h,v),n)
        push!(xs, multiRISE_links_only_direct(s, new_plaqs))
    end
    return xs
end

tanh2RG(K::Float64) = atanh(tanh(K)^2)
function RG_flow(K0::Float64, n_steps::Int)
    Ks = Float64[K0]
    K = K0
    for _ in 1:n_steps
        K = tanh2RG(K); push!(Ks, K)
    end
    return Ks
end

# checkerboard-based example
function checkerboard2_rg(samples::Matrix{Float64}, (h,v), n::Int)
    @assert n % 2 == 0 "n must be even"
    new_n = n ÷ 2
    N, _  = size(samples)
    plaqs = plaquettes(n, h, v)
    (h_new,v_new) = link_indices(new_n)
    plaqs_new     = plaquettes(new_n, h_new, v_new)
    samples_new   = Matrix{Float64}(undef, N, 2*new_n^2)
    for p in 1:length(plaqs)
        row = div(p-1, n)+1; col = mod(p-1, n)+1
        (t, _, _, l) = plaqs[p]
        Lp = samples[:,t] .* samples[:,l]
        ci = div(row-1, 2) + 1
        cj = div(col-1, 2) + 1
        idxc = isodd(p) ? h_new[(ci,cj)] : v_new[(ci,cj)]
        samples_new[:, idxc] = Lp
    end
    return samples_new, plaqs_new, (h_new, v_new), new_n
end

function run_plaquette_rg(samples_matrix, hmap, vmap, n, K, n_steps=3)
    K_values = [K]
    s_current = samples_matrix
    h_current, v_current = hmap, vmap
    n_current = n
    for step in 1:n_steps
        s_new, plaqs_new, (h_new, v_new), n_new = checkerboard2_rg(s_current, (h_current, v_current), n_current)
        K_new = multiRISE_links_only_direct(s_new, plaqs_new)
        push!(K_values, K_new)
        println("RG step $step: n = $n_current → $n_new, K = $(K_values[end-1]) → $K_new")
        s_current = s_new; h_current, v_current = h_new, v_new; n_current = n_new
    end
    return K_values
end

## 5. Wilson loops and string tension

We compute $\langle W(R,T)\rangle$ for rectangular loops and extract string tension via area-law:
$$
\sigma \approx -\frac{1}{A}\ln \langle W \rangle,\quad A=R\,T.
$$

In [ ]:
function compute_square_wilson_loops(samples::Matrix{Float64},
                                     hmap::Dict{Tuple{Int, Int}, Int},
                                     vmap::Dict{Tuple{Int, Int}, Int},
                                     max_size::Int)
    nconfigs, _ = size(samples)
    n = Int(sqrt(length(hmap)))
    loop_averages = Dict{Tuple{Int, Int}, Float64}()
    for R in 1:max_size
        loop_sum = 0.0; total_loops = 0
        for i in 1:n, j in 1:n
            top    = [hmap[(i, mod1(j+x-1, n))] for x in 1:R]
            right  = [vmap[(mod1(i+y-1, n), mod1(j+R, n))] for y in 1:R]
            bottom = [hmap[(mod1(i+R, n), mod1(j+x-1, n))] for x in R:-1:1]
            left   = [vmap[(mod1(i+y-1, n), j)] for y in R:-1:1]
            idx = vcat(top, right, bottom, left)
            for k in 1:nconfigs
                loop_sum += prod(samples[k, idx])
            end
            total_loops += 1
        end
        loop_averages[(R, R)] = loop_sum / (nconfigs * total_loops)
    end
    return loop_averages
end

function estimate_string_tension(wilson_dict::Dict{Tuple{Int, Int}, Float64})
    tension_estimates = Dict{Tuple{Int, Int}, Float64}()
    for ((R, T), W) in wilson_dict
        if R != T || W <= 0; continue; end
        A = R*T
        σ = -log(W) / A
        tension_estimates[(R, T)] = σ
    end
    mean_tension = mean(values(tension_estimates))
    return tension_estimates, mean_tension
end

function RG_with_K_and_string_tension(s::Matrix{Float64}, (h,v), n::Int, n_steps::Int, K0::Float64, max_loop_size::Int)
    sigmas = Float64[]
    s_current = s; (h_current, v_current) = (h, v); n_current = n
    for i in 0:n_steps
        wilson_loops = compute_square_wilson_loops(s_current, h_current, v_current, max_loop_size)
        _, avg_sigma = estimate_string_tension(wilson_loops)
        push!(sigmas, avg_sigma)
        println("Step $i: ⟨σ⟩ = $avg_sigma")
        if i < n_steps
            s_current, _, (h_current, v_current), n_current = coarse_plaquettes_overlap(s_current, (h_current, v_current), n_current)
        end
    end
    return sigmas
end

## 6. Autocorrelation & adaptive sampling helpers

These utilities estimate integrated autocorrelation time and suggest thinning. Also included: a parallel-chain sampler for quicker decorrelation (optional).

In [ ]:
function measure_actual_autocorrelation(samples::Matrix{Float64}, plaqlist::Vector{NTuple{4, Int}})
    energies = compute_plaquette_energy_per_sample(samples, plaqlist)
    max_lag = min(length(energies) ÷ 4, 1000)
    acf = autocor(energies, 0:max_lag)
    tau_int = 1.0
    for lag in 1:max_lag
        if acf[lag+1] < 0.05
            tau_int = 1.0 + 2.0 * sum(acf[2:lag+1]); break
        elseif lag == max_lag
            tau_int = 1.0 + 2.0 * sum(acf[2:end])
            println("ACF did not reach 0.05 by lag $max_lag")
        end
    end
    println("Lag-1 ACF: $(acf[2]); τ_int ≈ $tau_int")
    return tau_int, acf
end

## 7. Example usage (modular; run step-by-step)

**Sampling parameters** (edit as needed), then run Metropolis or Cluster, compute diagnostics, learn \(K\), apply RG, and estimate string tension.

In [ ]:
# --- Parameters (edit) ---
n = 16
K = 0.3
nsweeps = 200_000
nburn   = 20_000
step_interval = 10
num_samples = floor(Int, (nsweeps - nburn)/step_interval)

# Lattice
(hmap, vmap) = link_indices(n)
plaqs = plaquettes(n, hmap, vmap)
nvars = 2n*n

# --- Choose a sampler ---
samples_met = zeros(num_samples, nvars)
samples_clu = zeros(num_samples, nvars)

# Metropolis (reference)
# samples_met = metropolis_matrix(n, plaqs, K, nsweeps, nburn, samples_met; step_interval=step_interval)

# Cluster (faster decorrelation)
samples_clu = cluster_sampler(n, plaqs, K, nsweeps, nburn, samples_clu; step_interval=step_interval)

# --- Diagnostics ---
energies = compute_plaquette_energy_per_sample(samples_clu, plaqs)
histogram(energies, bins=80, xlabel="Plaquette energy", ylabel="Count", title="Energy histogram (Cluster)")
acf_vals = compute_and_plot_acf(energies; max_lag=1000)
tau = estimate_autocorrelation_length(acf_vals; cutoff=0.05)
@show tau

# --- Learn K from samples ---
K_rise  = multiRISE_links_only_direct(samples_clu, plaqs)
K_pl    = PL_links(samples_clu, plaqs)
K_ml    = ML_estimator_plaquette(samples_clu, plaqs)
@show K_rise K_pl K_ml

# --- RG flows (pick a scheme) ---
K_flow_overlap = RG_coarse_overlap(samples_clu, (hmap,vmap), n, 3, K)
K_flow_sum     = RG_coarse_sum(samples_clu, (hmap,vmap), n, 3, K)
K_flow_4x4     = RG_coarse_4x4(samples_clu, (hmap,vmap), n, 3, K)

# analytic flow (for comparison)
K_flow_analytic = RG_flow(K, 3)

# --- Wilson loops & string tension along RG ---
sigmas = RG_with_K_and_string_tension(samples_clu, (hmap, vmap), n, 3, K, 2)

# --- Plot flows ---
plot(0:3, K_flow_overlap, lw=2, marker=:circle, label="RG overlap")
plot!(0:3, K_flow_sum,     lw=2, marker=:square, label="RG sum")
plot!(0:3, K_flow_4x4,     lw=2, marker=:utriangle, label="RG 4x4")
plot!(0:3, K_flow_analytic, lw=2, marker=:diamond, label="analytic tanh^2 map")
xlabel!("RG step"); ylabel!("K"); title!("RG flows of K")

plot(0:3, sigmas, lw=2, marker=:circle, xlabel="RG step", ylabel="σ", title="String tension vs RG step", label="σ")